In [1]:
using Pkg
using Knet

In [3]:
include("Bimodal_Embedding.jl")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
 Installed FillArrays ─ v0.8.2
 Installed IterTools ── v1.3.0
 Installed Literate ─── v2.1.1
 Installed Plots ────── v0.27.1
  Updating `~/.julia/environments/v1.2/Project.toml`
  [3a865a2d] ↑ CuArrays v1.3.0 ⇒ v1.4.4
  [587475ba] ↓ Flux v0.9.0 ⇒ v0.8.3
  [c8e1da08] ↑ IterTools v1.2.0 ⇒ v1.3.0
  [98b081ad] ↑ Literate v2.1.0 ⇒ v2.1.1
  [91a5bcdd] ↑ Plots v0.27.0 ⇒ v0.27.1
  Updating `~/.julia/environments/v1.2/Manifest.toml`
  [c5f51814] ↑ CUDAdrv v3.1.0 ⇒ v4.0.3
  [be33ccc6] ↑ CUDAnative v2.4.0 ⇒ v2.5.4
  [3a865a2d] ↑ CuArrays v1.3.0 ⇒ v1.4.4
  [1a297f60] ↑ FillArrays v0.7.4 ⇒ v0.8.2
  [587475ba] ↓ Flux v0.9.0 ⇒ v0.8.3
  [0c68f7d7] ↑ GPUArrays v1.0.4 ⇒ v2.0.0
  [c8e1da08] ↑ IterTools v1.2.0 ⇒ v1.3.0
  [98b081ad] ↑ Literate v2.1.0 ⇒ v2.1.1
  [91a5bcdd] ↑ Plots v0.27.0 ⇒ v

┌ Info: Recompiling stale cache file /Users/deniz/.julia/compiled/v1.2/IterTools/hhnii.ji for IterTools [c8e1da08-722c-5040-9ed9-7db0dc04731e]
└ @ Base loading.jl:1240


 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


┌ Info: Recompiling stale cache file /Users/deniz/.julia/compiled/v1.2/Images/H8Vxc.ji for Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
└ @ Base loading.jl:1240


 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
695-element Array{Int32,1}
695-element Array{Tuple{Tuple{Array{UInt8,3},Array{UInt8,3}},String},1}
Train error before training is: 0.5871070615322231
Test error before training is: 0.5935088114212927


LoadError: MethodError: no method matching reluback(::Float64, ::Float32)
Closest candidates are:
  reluback(::Float64, !Matched::Knet.Bcasted{#s181} where #s181<:(KnetArray{Float64,N} where N)) at /Users/deniz/.julia/packages/Knet/HRYiN/src/binary.jl:126
  reluback(::T<:Number, !Matched::T<:Number) where T<:Number at /Users/deniz/.julia/packages/Knet/HRYiN/src/unary.jl:54
  reluback(!Matched::AutoGrad.Value{##408}, ::##409) where {##408, ##409} at none:0
  ...

In [6]:
Pkg.add("VideoIO"); using VideoIO

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [7]:
Pkg.add("Images"); using Images; Pkg.add("Colors"); using Colors

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [22]:
function GetPreProcessedImageArray(img, cropfromleft::Float64, cropfromright::Float64, cropfromtop::Float64, cropfrombottom::Float64, target_height::Int, target_width::Int)
    channels, height, width = size(img)
    img = img[:,Int(round(height*cropfromtop))+1:Int(round((1-cropfrombottom)*height)), Int(round(width*cropfromleft))+1:Int(round((1-cropfromright)*width))]
    img = imresize(img, (channels, target_height, target_width))
    permutedims(img, (2,3,1))
end

GetPreProcessedImageArray (generic function with 1 method)

In [23]:
io = VideoIO.open("montezuma2.mp4")
f = VideoIO.openvideo(io)

target_width = 160
target_height = 210
crop_from_top = 0.1
crop_from_bottom = 0.12
crop_from_left = 0.15
crop_from_right = 0.15


img = read(f)
img = GetPreProcessedImageArray(channelview(img), crop_from_left, crop_from_right, crop_from_top, crop_from_bottom, target_height, target_width)
frame_pairs = []

current_frame = img

while !eof(f)
    prev_frame = current_frame
    current_frame = read(f)
    current_frame = GetPreProcessedImageArray(channelview(current_frame), crop_from_left, crop_from_right, crop_from_top, crop_from_bottom,target_height, target_width)
    push!(frame_pairs, (prev_frame, current_frame))
end
close(f)
summary(frame_pairs)

"2486-element Array{Any,1}"

In [24]:
b = Knet.load("bimodalv6.jld2", "model")

bimodalEncoder(frame_head(Conv(P(Array{Float32,4}(5,5,3,32)), P(Array{Float32,4}(1,1,32,1)), NNlib.relu, 0, Knet.pool), Conv(P(Array{Float32,4}(5,5,32,32)), P(Array{Float32,4}(1,1,32,1)), NNlib.relu, 0, Knet.pool), Conv(P(Array{Float32,4}(4,4,32,64)), P(Array{Float32,4}(1,1,64,1)), NNlib.relu, 0, Knet.pool), Conv(P(Array{Float32,4}(3,3,64,64)), P(Array{Float32,4}(1,1,64,1)), NNlib.relu, 0, identity), Dense(P(Array{Float32,2}(10,47040)), P(Array{Float32,1}(10)), NNlib.relu, 0), Dense(P(Array{Float32,2}(10,10)), P(Array{Float32,1}(10)), NNlib.relu, 0)), sentence_head(Embed(P(Array{Float32,2}(12,21))), LSTM(input=12,hidden=10)))

In [25]:
both_frames = hcat(frame_pairs[1][1], frame_pairs[1][2])
summary(both_frames)
#colorview(RGB, both_frames)

"210×320×3 Array{N0f8,3} with eltype Normed{UInt8,8}"